## 2.语言模型N-grams##
__gram__, 这里原本可以译为质量的单位克(g),在这里可以理解为某种度量"单位", N-grams就是连续的N个词或者字符。为了获取所有的n-grams直观上就是用一个大小为N的滑动窗口对语料库中的文本进行划分，其实本质上可以理解为是N维向量的子空间，其中每个元素的定义域就是语料库。



### 预测下一个词###

现在我想再google上搜索和“语言模型”相关的内容，在我输入“语言模型”这几个字之后，搜索框会给出其他搜索待选项。这个过程被描述为给定一个序列的前几个词如何来预测下一个词。

![给定一个序列的前几个词预测下一个词](给定一个序列的前几个词预测下一个词.png)

很自然的，我们可以假设后面一个待预测的词与前面已经出现的词有关，需要从语料库中找到出现概率最大的词。这样就相对于计算如下的条件概率：

$$p(w_i|w_{i-1}...w_{1}) = \frac{p(w_iw_{i-1}...w_{1})}{p(w_{i-1}...w_{1})} = \frac{c(w_iw_{i-1}...w_{1})}{c(w_{i-1}...w_{1})}$$

根据条件概率公式可以转换成联合概率的表达形式，实际计算中只需要统计词片段序列$w_iw_{i-1}...w_{1}$ 和 $w_{i-1}...w_{1}$在语料库出现的次数。

### 预测序列出现的概率 ###

有些场景下我们想要自动生成某个句子$w=(w_1...w_k)$，这相当于学习$w$的联合概率分布,根据链式法则有:
$$p(w) = p(w_1)p(w_2|w_1)p(w_3|w_1w_2)p(w_k|w_1w_2...w_{k-1})$$
可以看到任意一个词的出现都前面出现过的所有词相关，为了简化计算可以利用马尔可夫假设，第i个词的出现只和紧挨着的之前出现过的N-1个词相关,这是一个$n-1$阶的马尔可夫模型：
$$p(w_i|w_1...w_{i-1}) = p(w_i|w_{i-n+1}w_{i-1})$$

于是当n=1时，我们得到一个一元模型(unigram model):
$$p(w_1w_2...w_i) = p(w_1)p(w_2)...p(w_i)$$
unigram任务序列中每个词都时独立的，其中某个词出现的概率和前面的词无关。

当n=2时，我们得到一个二元模型(bigram model):

$$p(w_1w_2...w_i) = p(w_1)p(w_1|w_2)p(w_2|w_3)...p(w_i|w_{i-1})$$
序列中每个词出现的概率都只和前面紧挨的一个词有关

当n=3时，我们得到一个三元模型(trigram model):

$$p(w_1w_2...w_i) = p(w_1)p(w_1|w_2w_3)p(w_2|w_3w_4)...p(w_i|w_{i-1}w_{i-2})$$
序列中每个词出现的概率都只和前面紧挨的两个词有关。


### 句子中的开始符和结束符###

#### 开始符 #### 
考虑一个bigram模型生成句子：
$$p(w) = p(w_1)p(w_2|w_1)...p(w_k|w_{k-1})$$
在实际情况中绝大多数词出现在开始位置的概率和出现在其他地方的概率肯定时不一样的，所以不能直接使用$p(w_1)$,引入一个开始符__start__来解决上述问题,用来$p(w_1|start)$代表词$w_1$出现在开始的概率。
$$p(w) = p(w_1|start)p(w_2|w_1)...p(w_k|w_{k-1})$$

#### 结束符 ####

在n-grams中使用不同的n得到的向量特征空间不同，但是具体的某个n之下，向量出现的概率是归一化的。例如$n=1$, $\sum_{w_i \in V}p(w_i)=1$。对于$n=2$, 则有 $\sum_{w_i \in V, w_j \in V}p(w_ip_j)=1$,其中V是语料库。
对于不同长度的序列，它们是分别归一化的。为了统一归一化，即是让不同长度的序列出现的概率之和为1，引入一个结束符号__end__。于是
$$p(w) = p(w_1|start)p(w_2|w_1)...p(w_k|w_{k-1})p(end|w_k)$$